In [1]:
from gurobipy import *
import pandas as pd
import numpy as np
import csv

In [200]:
df = pd.read_excel("../data/焚化廠資料.xlsx")
df

,焚化廠名稱,縣市,年份,其他成本,勞動成本,焚化處理量,設計處理量,設計發電量,實際發電量,二氧化碳當量
0,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2020,960330.0,1033323.0,182263.4,600,14700000,99631.87,136697.550
1,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2011,329010.0,468548.0,199732.2,600,14700000,107838.70,149799.150
2,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2019,846509.0,999635.0,202111.2,600,14700000,107890.40,151583.400
3,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2012,371861.0,518953.0,202728.5,600,14700000,110156.70,152046.375
4,宜蘭縣利澤垃圾資源回收(焚化)廠,宜蘭縣,2013,411821.0,587162.0,205014.9,600,14700000,110901.60,153761.175
...,...,...,...,...,...,...,...,...,...,...
235,臺南市城西垃圾焚化廠,臺南市,2014,1085896.0,1285597.5,219334.7,900,14300000,91008.24,164501.025
236,臺南市城西垃圾焚化廠,臺南市,2016,1196041.0,1387306.0,219850.9,900,14300000,92339.52,164888.175
237,臺南市城西垃圾焚化廠,臺南市,2015,1096999.5,1299537.5,221728.8,900,14300000,93911.04,166296.600
238,臺南市城西垃圾焚化廠,臺南市,2011,835958.0,999123.0,222657.9,900,14300000,91025.38,166993.425


In [201]:
yearly_df = df.groupby('年份')

In [202]:
# Sets
Years = list(range(2011, 2021))
K = list(range(24))  # number of DMUs
I_V = [0,1,2]  # variable input amount
I_F = [0,1]  # fixed input amount
J = [0]  # output 
M = [0]  # bad output
years = list(range(1,11))
dmus = list(range(1,25))

### Create a production data object to store the production data including:

In [203]:
# Object of production data
class prod_data:
    """
    Attr:
    * x[i][r]: Input i of DMU r
    * y[j][r]: Output j of DMU r
    * b[k][r]: Bad output k of DMU r
    """
    def __init__(self, X_V, X_F, Y, B):
        self.x_v = X_V
        self.x_f = X_F
        self.y = Y
        self.b = B

In [204]:
yearly_prod_data = {}
for year in Years:
    # Parameters
    X_V = [
        list(yearly_df.get_group(year)["其他成本"].values),
        list(yearly_df.get_group(year)["勞動成本"].values),
        list(yearly_df.get_group(year)["焚化處理量"].values)
    ]
    X_F = [
        list(yearly_df.get_group(year)["設計處理量"].values),
        list(yearly_df.get_group(year)["設計發電量"].values)
    ]
    Y = [list(yearly_df.get_group(year)["實際發電量"].values)]
    B = [list(yearly_df.get_group(year)["二氧化碳當量"].values)]
    yearly_prod_data[year] = prod_data(X_V, X_F, Y, B)

In [205]:
yearly_prod_data[2020].b

[[136697.55,
  188625.59999999998,
  109951.72499999999,
  289382.475,
  284787.75,
  245548.34999999998,
  140791.72499999998,
  282796.275,
  133405.125,
  284376.525,
  151773.825,
  188916.675,
  164930.09999999998,
  47429.587499999994,
  201153.15000000002,
  190617.52500000002,
  136925.625,
  196591.8,
  203855.32499999998,
  101225.09999999999,
  162133.8,
  267403.5,
  188394.52500000002,
  123697.65000000001]]

## 方法論
我們希望將假設不完全競爭市場的 decentralize model 套用在台灣的焚化爐上，並擴展到跨時間維度。在假設各個發電廠會最大化自己在所有時間區段的總盈餘情況下計算 Nash 均衡。

### 計算 $\Delta B_r^-$ 和 $\Delta B_r^+$
$$
\max \sum_{k\in K}\lambda_{kr}B_{k} \\
\text{s.t.} \sum_{k\in K}\lambda_{kr}X_{ik} \leq X_{ir}, \quad \forall i \in I_f \\
\sum_{k\in K}\lambda_{kr}X_{ik} \leq x_{ir}, \quad \forall i \in I_v \\
\sum_{k\in K}\lambda_{kr}Y_{k} \geq y_r \\
\lambda_{kr}, x_{ir}, y_r \geq 0, \forall k \in K, i \in I_v
$$

In [117]:
def initial_allocation(year, production_data): 
    X_V = production_data[year].x_v
    X_F = production_data[year].x_f
    Y = production_data[year].y
    B = production_data[year].b
    # Save CRS for each DMU
    crs = [0 for r in K]

    for r in K:
        md11 = Model('model')
        l = md11.addVars(K, vtype=GRB.CONTINUOUS, name='l')
        x = md11.addVars(I_V, vtype=GRB.CONTINUOUS, name='x')
        y = md11.addVars(J, vtype=GRB.CONTINUOUS, name='y')  

        md11.setObjective(
            quicksum(l[k] * B[0][k] for k in K), GRB.MAXIMIZE
        )

        md11.addConstrs(
            quicksum(l[k] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )

        md11.addConstrs(
            quicksum(l[k] * X_V[i][k] for k in K) <= x[i]
            for i in I_V
        )

        md11.addConstrs(
            quicksum(l[k] * Y[j][k] for k in K) >= y[j]
            for j in J
        )

        md11.params.LogToConsole = 0
        # md11.params.NonConvex = 2
        md11.optimize() 

        print(df.loc[r,"焚化廠名稱"])
        for var in md11.getVars():
            print(f"{var.VarName}, {var.X}")

        # to load crs
        crs[r] = md11.objVal
    return crs

In [120]:
initial_permits = {}
year = 2020
initial_permits[year] = initial_allocation(year, yearly_prod_data)

宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 1.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
x[0], 960330.0
x[1], 1033323.0
x[2], 182263.4
y[0], 0.0
宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 1.5
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
x[0], 1440495.0
x[1], 1549984.5
x[2], 273395.1
y[0], 0.0
宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 0.7885483014861996
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.08957006369426741
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
x[0], 787873.2185509554
x[1], 884349.648686

In [121]:
initial_permits

{2020: [136697.55,
  205046.32499999998,
  112040.89204319267,
  307569.4875,
  307569.4875,
  307569.4875,
  205046.32499999998,
  410092.64999999997,
  136697.55,
  307569.4875,
  174025.63493730093,
  262703.99281200237,
  205046.32499999998,
  47429.587499999994,
  205046.32499999998,
  205046.32499999998,
  167363.4733130971,
  205046.32499999998,
  205046.32499999998,
  122346.31341358555,
  244934.33889828817,
  410092.64999999997,
  205046.32499999998,
  172776.47963276273]}

# 計算碳排放權的分配
### Decentralized model

In [235]:
def decentralized_allocation(production_data):
    X_V = production_data.x_v
    X_F = production_data.x_f
    Y = production_data.y
    B = production_data.b

    # Build Model
    md2 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    x = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
    y = md2.addVars(K, vtype=GRB.CONTINUOUS, name='y') 
    b = md2.addVars(K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 
    phi1 = md2.addVars(I_F, K, vtype=GRB.CONTINUOUS, name='phi1', lb=LB)
    phi2 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='phi2', lb=LB)
    phi3 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi3', lb=LB)
    phi4 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi4', lb=LB)
    phi5 = md2.addVar(vtype=GRB.CONTINUOUS, name='phi5', lb=LB)
    phi6 = md2.addVar(vtype=GRB.CONTINUOUS, name='phi6', lb=LB)
    phi7 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi7', lb=LB)
    phi8 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='phi8', lb=LB)
    a1 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='a1')
    a2 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a2')
    a3 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a3')
    a4 = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='a4')
    a5 = md2.addVars(I_F, K, vtype=GRB.CONTINUOUS, name='a5')
    a6 = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='a6')
    a7 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a7')
    a8 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a8')
    a9 = md2.addVar(vtype=GRB.CONTINUOUS, name='a9')
    a10 = md2.addVar(vtype=GRB.CONTINUOUS, name='a10')
    a11 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a11')
    a12 = md2.addVars(K, vtype=GRB.CONTINUOUS, name='a12')

    md2.setObjective(
        quicksum(a1[i, r] for i in I_V for r in K) 
        + quicksum(a2[r] for r in K)
        + quicksum(a3[r] for r in K)
        + quicksum(a4[k, r] for k in K for r in K)
        + quicksum(a5[i, r] for i in I_F for r in K)
        + quicksum(a6[i, r] for i in I_V for r in K)
        + quicksum(a7[r] for r in K)
        + quicksum(a8[r] for r in K)
        + a9 + a10
        + quicksum(a11[r] for r in K)
        + quicksum(a12[r] for r in K)
        , GRB.MINIMIZE
    )

    ## KKT 
    md2.addConstrs(
        (
            1 - (phi2[i,r]) 
        ) - a1[i, r] == 0
        for i in I_V for r in K
    )

    md2.addConstrs(
        (P_Y - tao * (
            quicksum(y[k] for k in K) + Y_bar   #Y_hat
        ) - tao * y[r] - phi3[r] + phi5) 
        - a2[r] == 0
        for r in K
    )

    md2.addConstrs(
        epsolon - phi4[r] + phi6 + phi7[r] - phi8[r] 
        - a3[r] == 0
        for r in K
    )

    md2.addConstrs(

        (
            - quicksum(phi1[i,r] * X_F[i][k] for i in I_F)  
            - quicksum(phi2[i,r] * X_V[i][k] for i in I_V)
            + phi3[r] * Y[0][k]
            - phi4[r] * B[0][k]
        ) - a4[k, r] == 0
        for r in K for k in K
    )

    md2.addConstrs(
        
        ( 
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) - X_F[i][r] 
        ) - a5[i, r] == 0
        for i in I_F for r in K
    )

    md2.addConstrs(

        ( 
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) - x[i,r] 
        ) - a6[i, r] == 0
        for i in I_V for r in K
    )

    md2.addConstrs(

        ( 
            y[r] - quicksum(_lambda[k,r] * Y[0][k] for k in K) 
        ) - a7[r] == 0
        for r in K
    )

    md2.addConstrs(
        quicksum(_lambda[k,r] * B[0][k] for k in K) - B[0][r] + b[r]
        - a8[r] == 0
        for r in K
    )

    md2.addConstr(

        (Demand - quicksum(y[r] for r in K) )
        - a9 == 0
    )

    md2.addConstr(
        Cap - quicksum(b[r] for r in K) - a10 == 0
    )

    md2.addConstrs(
        phi7[r] * 
        ( 
            B_minus[r] - b[r]
        ) - a11[r] == 0
        for r in K
    )

    md2.addConstrs(
        phi8[r] *
        ( 
            b[r] - B_plus[r]
        ) - a12[r] == 0
        for r in K
    )


    # md2.write("1.lp")
    # md2.params.LogToConsole = 0
    md2.params.NonConvex = 2
    md2.optimize() 

    # Return variables as objects
    result = prod_data(
        X_V=[[0 for _ in K] for _ in I_V],
        X_F=X_F,
        Y=[[0 for _ in K] for _ in J],
        B=[[0 for _ in K] for _ in M]
    )
    x_sol = md2.getAttr('X',x)
    y_sol = md2.getAttr('X',y)
    b_sol = md2.getAttr('X',b)
    permits = [0 for _ in K]
    for r in K:
        for i in I_V:
            result.x_v[i][r] = x_sol[(i,r)]
        result.y[0][r] = y_sol[(r)]
        result.b[0][r] = B[0][r] - b_sol[(r)]
        permits[r] = b_sol[(r)]
    revenue = md2.objVal

    for var in md2.getVars():
        print(f"{var.VarName}, {var.X}")

    # # Save variables as csv
    # csv_file = open(f"../results/permit_{year}.csv", "a")
    # for var in md2.getVars(): 
    #     writer = csv.writer(csv_file)
    #     writer.writerow([var.VarName, var.X])
    # csv_file.close()

    return result, permits, revenue


### Lozano model

In [242]:
def lozano_allocation(production_data): 
    X_V = production_data.x_v
    X_F = production_data.x_f
    Y = production_data.y
    B = production_data.b


    # Phase1: Maximize y
    md1 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md1.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    _gamma = md1.addVars(J, vtype=GRB.CONTINUOUS, name='gamma')
    x = md1.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
    y = md1.addVars(J, K, vtype=GRB.CONTINUOUS, name='y') 
    b = md1.addVars(M, K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 

    md1.setObjective(
        1/len(J) * quicksum(_gamma[j] for j in J), GRB.MAXIMIZE
    )
    for r in K:
        md1.addConstrs(
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= x[i, r]
            for i in I_V
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * Y[j][k] for k in K) >= y[j, r]
            for j in J
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * B[m][k] for k in K) == b[m, r]
            for m in M
        )
        md1.addConstrs(
            x[i, r] <= X_V[i][r]
            for i in I_V
        )
        md1.addConstrs(
            y[j, r] >= Y[j][r]
            for j in J
        )
    md1.addConstrs(
        quicksum(y[j, r] for r in K) >= _gamma[j] * quicksum(Y[j][r] for r in K)
        for j in J
    )
    md1.addConstrs(
        quicksum(b[m, r] for r in K) <= quicksum(B[m][r] for r in K)
        for m in M
    )

    md1.params.LogToConsole = 0
    md1.optimize() 


    # Phase2: Minimize b
    y_sol = md1.getAttr('X',y)
    Y_star = [[y_sol[(0,r)] for r in K]]

    md2 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md2.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    _theta = md2.addVars(M, vtype=GRB.CONTINUOUS, name='theta', ub=1)
    x = md2.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
    y = md2.addVars(J, K, vtype=GRB.CONTINUOUS, name='y') 
    b = md2.addVars(M, K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 

    md2.setObjective(
        1/len(M) * quicksum(_theta[m] for m in M), GRB.MINIMIZE
    )
    for r in K:
        md2.addConstrs(
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )
        md2.addConstrs(
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= x[i, r]
            for i in I_V
        )
        md2.addConstrs(
            quicksum(_lambda[k,r] * Y[j][k] for k in K) >= y[j, r]
            for j in J
        )
        md2.addConstrs(
            quicksum(_lambda[k,r] * B[m][k] for k in K) == b[m, r]
            for m in M
        )
        md2.addConstrs(
            x[i, r] <= X_V[i][r]
            for i in I_V
        )
        md2.addConstrs(
            y[j, r] >= Y[j][r]
            for j in J
        )

    md2.addConstrs(
        quicksum(y[j, k] for k in K) >= quicksum(Y_star[j][k] for k in K)
        for j in J
    )
    
    md2.addConstrs(
        quicksum(b[m, r] for r in K) <= _theta[m] * quicksum(B[m][r] for r in K)
        for m in M
    )
    # md2.addConstrs(
    #     quicksum(b[m, r] for r in K) == Cap  # 加一個限制，使得碳排放總量 == Cap 要求的量
    #     for m in M
    # )

    md2.params.LogToConsole = 0
    md2.optimize() 

    # Phase3: Minimize x
    theta_sol = md2.getAttr('X',_theta)
    theta_star = [theta_sol[(m)] for m in M]
    print("theta_star", theta_star)

    md3 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md3.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    _beta = md3.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='beta', ub=1)
    x = md3.addVars(I_V, K, vtype=GRB.CONTINUOUS, name='x')
    y = md3.addVars(J, K, vtype=GRB.CONTINUOUS, name='y') 
    b = md3.addVars(M, K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 

    md3.setObjective(
        1/len(I_V) * quicksum(_beta[i,k] for i in I_V for k in K), GRB.MINIMIZE
    )
    for r in K:
        md3.addConstrs(
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )
        md3.addConstrs(
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= x[i, r]
            for i in I_V
        )
        md3.addConstrs(
            quicksum(_lambda[k,r] * Y[j][k] for k in K) >= y[j, r]
            for j in J
        )
        md3.addConstrs(
            quicksum(_lambda[k,r] * B[m][k] for k in K) == b[m, r]
            for m in M
        )
        md3.addConstrs(
            x[i, r] == _beta[i,r] * X_V[i][r]
            for i in I_V
        )
        md3.addConstrs(
            y[j, r] >= Y[j][r]
            for j in J
        )

    md3.addConstrs(
        quicksum(y[j, r] for r in K) >= quicksum(Y_star[j][r] for r in K)
        for j in J
    )

    md3.params.LogToConsole = 0
    md3.optimize() 

    """Return variables as objects"""
    result = prod_data(
        X_V=[[0 for _ in K] for _ in I_V],
        X_F=X_F,
        Y=[[0 for _ in K] for _ in J],
        B=[[0 for _ in K] for _ in M]
    )
    x_sol = md3.getAttr('X',x)
    y_sol = md3.getAttr('X',y)
    b_sol = md3.getAttr('X',b)
    permits = [0 for _ in K]
    for r in K:
        for i in I_V:
            result.x_v[i][r] = x_sol[(i,r)]
        for j in J:
            result.y[0][r] = y_sol[(j,r)]
        for m in M:
            result.b[0][r] = b_sol[(m,r)]
            permits[r] = production_data.b[0][r] - b_sol[(m,r)]
    revenue = md3.objVal

    for var in md1.getVars():
        print(f"{var.VarName}, {var.X}")

    return result, permits, revenue

## Feng Model

In [239]:
def feng_allocation(production_data): 
    X_V = production_data.x_v
    X_F = production_data.x_f
    Y = production_data.y
    B = production_data.b

    md1 = Model('model')
    LB = -GRB.INFINITY
    _lambda = md1.addVars(K, K, vtype=GRB.CONTINUOUS, name='lambda')
    b = md1.addVars(M, K, vtype=GRB.CONTINUOUS, name='b', lb=LB) 

    md1.setObjective(
        quicksum(
            _lambda[k, r] * Y[j][k] for j in J for k in K for r in K
        ), GRB.MAXIMIZE
    )
    for r in K:
        md1.addConstrs(
            quicksum(_lambda[k,r] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * X_V[i][k] for k in K) <= X_V[i][r]
            for i in I_V
        )
        md1.addConstrs(
            quicksum(_lambda[k,r] * B[m][k] for k in K) == B[m][r] - b[m, r]
            for m in M
        )
        md1.addConstr(
            b[0, r] >= B_minus[r]
        )
        md1.addConstr(
            b[0, r] <= B_plus[r]
        )
    md1.addConstr(
        quicksum(b[0, k] for k in K) == Cap
    )

    # md1.params.LogToConsole = 0
    md1.optimize() 

    """Return variables as objects"""
    result = prod_data(
        X_V=[[0 for _ in K] for _ in I_V],
        X_F=X_F,
        Y=[[0 for _ in K] for _ in J],
        B=[[0 for _ in K] for _ in M]
    )
    # x_sol = md1.getAttr('X',x)
    # y_sol = md1.getAttr('X',y)
    b_sol = md1.getAttr('X',b)
    permits = [0 for _ in K]
    for r in K:
        # for i in I_V:
        #     result.x_v[i][r] = x_sol[(i,r)]
        # for j in J:
        #     result.y[0][r] = y_sol[(j,r)]
        for m in M:
            result.b[m][r] = B[m][r] - b_sol[(m,r)]
            permits[r] = b_sol[(m,r)]
    result.x_v = X_V
    result.x_y = Y
    total_output = md1.objVal

    for var in md1.getVars():
        print(f"{var.VarName}, {var.X}")

    return result, permits, total_output

## 利用 Fare Distance Model to evaluate efficiency of the current permit allocation method.
x, y 是原本給定的參數，只有 b 為分配後的結果

In [238]:
def fare_efficiency(production_data): 
    X_V = production_data.x_v
    X_F = production_data.x_f
    Y = production_data.y
    B = production_data.b
    # Save CRS for each DMU

    efficiency = [0 for r in K]

    for r in K:
        md11 = Model('model')
        l = md11.addVars(K, vtype=GRB.CONTINUOUS, name='l')
        _theta = md11.addVar(vtype=GRB.CONTINUOUS, name='theta')
        _epsilon = md11.addVar(vtype=GRB.CONTINUOUS, name='epsilon', ub=1)

        md11.setObjective(
            _theta, GRB.MINIMIZE
        )

        md11.addConstrs(
            quicksum(l[k] * X_F[i][k] for k in K) <= X_F[i][r]
            for i in I_F
        )

        md11.addConstrs(
            quicksum(l[k] * X_V[i][k] for k in K) <= _epsilon * X_V[i][r]
            for i in I_V
        )

        md11.addConstrs(
            quicksum(l[k] * Y[j][k] for k in K) >= Y[j][r]
            for j in J
        )

        md11.addConstrs(
            quicksum(l[k] * B[m][k] for k in K) == _theta * B[m][r]
            for m in M
        )

        md11.addConstr(
            quicksum(l[k] for k in K) == _epsilon
        )

        md11.params.LogToConsole = 0
        md11.optimize() 

        print(df.loc[r,"焚化廠名稱"])
        for var in md11.getVars():
            print(f"{var.VarName}, {var.X}")

        # to load crs
        efficiency[r] = md11.objVal
    return efficiency

In [243]:
year = 2020    

B_minus = (np.array(B[0])- np.array(initial_permits[year])).tolist()  #減碳下限
# B_minus = [0 for k in K]
B_plus = B[0]   #減碳上限為原本的排放量
Cap = sum(B[0]) * 0.01    #Cap of trade
Demand = 2000000  #Demand of electricity
# P_X = 590
P_Y = 4 * 1e7
tao = 123
Y_bar = 10975
epsolon = 0

lozano_prod, lozano_permits, _  = lozano_allocation(yearly_prod_data[2020])
feng_prod, feng_permits, _  = feng_allocation(yearly_prod_data[2020])
decentralized_prod, decentralized_permits, revenue  = decentralized_allocation(yearly_prod_data[2020])

theta_star [0.9597513308428073]
lambda[0,0], 0.0
lambda[0,1], 0.0
lambda[0,2], 0.0
lambda[0,3], 0.0
lambda[0,4], 0.0
lambda[0,5], 0.0
lambda[0,6], 0.0
lambda[0,7], 0.0
lambda[0,8], 0.0
lambda[0,9], 0.0
lambda[0,10], 0.0
lambda[0,11], 0.0
lambda[0,12], 0.0
lambda[0,13], 0.0
lambda[0,14], 0.0
lambda[0,15], 0.0
lambda[0,16], 0.0
lambda[0,17], 0.0
lambda[0,18], 0.0
lambda[0,19], 0.0
lambda[0,20], 0.0
lambda[0,21], 0.0
lambda[0,22], 0.0
lambda[0,23], 0.0
lambda[1,0], 0.0
lambda[1,1], 0.0
lambda[1,2], 0.0
lambda[1,3], 0.0
lambda[1,4], 0.0
lambda[1,5], 0.0
lambda[1,6], 0.0
lambda[1,7], 0.0
lambda[1,8], 0.0
lambda[1,9], 0.0
lambda[1,10], 0.0
lambda[1,11], 0.0
lambda[1,12], 0.0
lambda[1,13], 0.0
lambda[1,14], 0.0
lambda[1,15], 0.0
lambda[1,16], 0.0
lambda[1,17], 0.0
lambda[1,18], 0.0
lambda[1,19], 0.0
lambda[1,20], 0.0
lambda[1,21], 0.0
lambda[1,22], 0.0
lambda[1,23], 0.0
lambda[2,0], 0.0
lambda[2,1], 0.0
lambda[2,2], 0.0
lambda[2,3], 0.0
lambda[2,4], 0.0
lambda[2,5], 0.0
lambda[2,6], 0.0
lambd

In [244]:
display(lozano_permits, feng_permits, decentralized_permits)

[12069.907068680477,
 0.0,
 3513.9889380531095,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 5.820766091346741e-11,
 20079.67699115048,
 -2.9103830456733704e-11,
 0.0,
 26448.725289823007,
 0.0,
 2.9103830456733704e-11,
 19663.71238938054,
 0.0,
 0.0,
 48484.2922965078,
 47695.61714488173,
 0.0,
 0.0,
 -1.4551915228366852e-11]

[12069.907068680483,
 0.0,
 2733.591479064322,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 29410.6143272552,
 0.0,
 -7.275957614183426e-12,
 0.0]

[0.0,
 2818.691603600979,
 -2089.1670431941748,
 43467.00120048225,
 40924.39882928133,
 17426.105409935117,
 0.0,
 5270.940384611487,
 6170.578987389803,
 38702.60615807772,
 -21785.61846922338,
 -58614.47366821766,
 23994.50680272281,
 0.0,
 45599.63849334419,
 4530.2523084282875,
 -30437.848313093185,
 4323.637087136507,
 62829.20799319446,
 0.0,
 -80256.94981969893,
 -12168.182040646672,
 2587.6166035979986,
 -49078.829632759094]

In [227]:
feng_prod.x_v

[[960330.0,
  1308329.0,
  895071.0,
  3730386.0,
  771524.75,
  771524.75,
  771524.75,
  771524.75,
  287219.0,
  2569321.6666666665,
  2569321.6666666665,
  2569321.6666666665,
  473901.0,
  341706.0,
  639013.0,
  1381022.0,
  1502592.3333333333,
  1502592.3333333333,
  1502592.3333333333,
  283331.6666666667,
  283331.6666666667,
  283331.6666666667,
  1090514.5,
  1090514.5],
 [1033323.0,
  1830292.0,
  1008138.0,
  5417133.0,
  1083453.0,
  1083453.0,
  1083453.0,
  1083453.0,
  1096798.0,
  3002925.6666666665,
  3002925.6666666665,
  3002925.6666666665,
  976738.0,
  776203.0,
  871609.0,
  1623739.0,
  1898015.3333333333,
  1898015.3333333333,
  1898015.3333333333,
  2409203.3333333335,
  2409203.3333333335,
  2409203.3333333335,
  1429848.5,
  1429848.5],
 [182263.4,
  251500.8,
  146602.3,
  385843.3,
  379717.0,
  327397.8,
  187722.3,
  377061.7,
  177873.5,
  379168.7,
  202365.1,
  251888.9,
  219906.8,
  63239.45,
  268204.2,
  254156.7,
  182567.5,
  262122.4,
  271807

In [234]:
lozano_prod

In [230]:
decentralized_prod.x_v

[[960330.0,
  1657950.2998735378,
  787873.2185509554,
  1851303.570506828,
  1795525.9739414288,
  1212254.2341777594,
  771524.7500002036,
  1492978.4145299145,
  1127035.1496043522,
  1838579.7924959825,
  1239704.3273985898,
  2011149.2709909738,
  795895.5514793219,
  341706.0,
  1056323.9717964248,
  1662027.3021920116,
  1191081.3953025478,
  1601997.7323145103,
  811229.9883651874,
  283331.6666666667,
  1784769.9433554641,
  1579370.898804723,
  1657950.2998735378,
  1226908.7639331212],
 [1033323.0,
  2028040.2697992416,
  884349.6486863056,
  2408953.186602547,
  2282663.9509000853,
  1661516.456658895,
  1083453.0000002775,
  2302880.3931623935,
  1309934.2048714147,
  2360736.474155426,
  1945923.0416785185,
  3241875.2846392943,
  1203114.8982831228,
  776203.0,
  1361336.4543498738,
  2026103.8651464856,
  2017510.4902468154,
  1879347.063282075,
  1278407.778101717,
  2409203.3333333335,
  3844804.2329934663,
  2571161.724528158,
  2028040.2697992416,
  1950352.06070859

In [246]:
# original_efficiency = {}
# decentralized_efficiency = {}
""" 計算效率 """
original_efficiency = fare_efficiency(yearly_prod_data[2020])
lozano_efficiency = fare_efficiency(lozano_prod)
feng_efficiency = fare_efficiency(feng_prod)
decentralized_efficiency = fare_efficiency(decentralized_prod)

宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.09397537252943561
l[6], 0.0
l[7], 0.0
l[8], 0.5987269884253072
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.13185562095818823
l[14], 0.05449413009651386
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
theta, 0.879052112009445
epsilon, 0.879052112009445
宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 0.0
l[1], 0.0
l[2], 0.0
l[3], 0.0
l[4], 0.0
l[5], 0.39790373481040753
l[6], 0.0
l[7], 0.0
l[8], 0.41016986580936016
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0.0
l[23], 0.0
theta, 0.8080736006197676
epsilon, 0.8080736006197677
宜蘭縣利澤垃圾資源回收(焚化)廠
l[0], 0.0
l[1], 0.0
l[2], 1.0
l[3], 0.0
l[4], 0.0
l[5], 0.0
l[6], 0.0
l[7], 0.0
l[8], 0.0
l[9], 0.0
l[10], 0.0
l[11], 0.0
l[12], 0.0
l[13], 0.0
l[14], 0.0
l[15], 0.0
l[16], 0.0
l[17], 0.0
l[18], 0.0
l[19], 0.0
l[20], 0.0
l[21], 0.0
l[22], 0

In [255]:
pd.DataFrame([original_efficiency, lozano_efficiency, feng_efficiency, decentralized_efficiency], index=["Original","Lozano", "Feng", "Nash CRS"]).T

,0,1,2,3
0,0.879052,1.0,0.0,1.000000
1,0.808074,1.0,0.0,0.638994
2,1.000000,1.0,0.0,1.000000
3,1.000000,1.0,0.0,0.781847
4,1.000000,1.0,0.0,0.748025
5,1.000000,1.0,0.0,1.000000
6,0.450307,1.0,0.0,0.543375
7,0.855392,1.0,0.0,0.728933
8,1.000000,1.0,0.0,0.684363
9,0.538657,1.0,0.0,0.782615


In [61]:
pd.DataFrame(decentralized_efficiency).to_csv("../results/decentralized_efficiency.csv")